# Diplomatura en ciencia de datos, aprendizaje automático y sus aplicaciones - Edición 2023 - FAMAF (UNC)

## Análisis exploratorio y curación de datos

### Trabajo práctico entregable - Grupo 22 - Parte 1

**Integrantes:**
- Chevallier-Boutell, Ignacio José
- Ribetto, Federico Daniel
- Rosa, Santiago

**Seguimiento:** Meinardi, Vanesa

---

## Librerías

In [ ]:
import io
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy

pd.set_option('display.max_rows', 1000) # cambiar el número de filas que se mostrarán usando display.max_rows.
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.options.mode.chained_assignment = None  # default='warn'

sns.set_context('talk')
sns.set_theme(style='white')

## Lectura del dataset

bla bla

In [ ]:
url = 'https://raw.githubusercontent.com/DiploDatos/AnalisisyVisualizacion/master/sysarmy_survey_2022_processed.csv'
df = pd.read_csv(url)
total_ans = len(df) # cantidad de respuestas en tel dataset

---
# Ejercicio 1 - 